In [3]:
import pandas as pd
import numpy as np
import time

# 读取女性评分
with open("Scores_for_Women.txt", "r") as f:
    w_to_m_data = f.readlines()[1:]

# 转化为矩阵 w_to_m_score_matrix[i][j] 为 女性i对男性j的评分
w_to_m_Scores = []    
for i in range(len(w_to_m_data)):
    w_to_m_Scores.append([float(ii) for ii in w_to_m_data[i].strip('\n').split(' ')[:-1]])
    
w_to_m_score_matrix = np.array(w_to_m_Scores)

# 读取女性评分
with open("Scores_for_Men.txt", "r") as f:
    m_to_w_data = f.readlines()[1:]

# 转化为矩阵 m_to_w_score_matrix[i][j] 为 男性i对女性j的评分    
m_to_w_Scores = []    
for i in range(len(m_to_w_data)):
    m_to_w_Scores.append([float(ii) for ii in m_to_w_data[i].strip('\n').split(' ')[:-1]])
    
m_to_w_score_matrix = np.array(m_to_w_Scores)


# 读取拒绝列表
with open("Reject_List.txt", "r") as f:
    reject_data = f.readlines()[1:]

# 转化为列表 
reject_list = []    
for i in range(len(reject_data)):
    reject_list.append([int(ii) for ii in reject_data[i].strip('\n').split(' ')])
    
# print('reject_list', reject_list)

W_dict = {} # 女性可接受词典
M_dict = {} # 男性可接受词典

for i in range(68):
    W_dict[i] = [i for i in range(68)]
    M_dict[i] = [i for i in range(68)]

for j in reject_list: # 按拒绝列表移除不接受的用户
    W_dict[j[0]].remove(j[1])
    M_dict[j[1]].remove(j[0])

In [5]:
def gale_shapley_modified_v2(m_to_w_score_matrix, w_to_m_score_matrix, M_dict, W_dict):
    num_m, num_w = len(m_to_w_score_matrix), len(w_to_m_score_matrix)
    
    matched_men = [-1] * num_m
    matched_women = [-1] * num_w

    # 男生向女生匹配的顺序
    proposals_order = []
    for m in range(num_m):
        preferences = sorted(W_dict[m], key=lambda w: -m_to_w_score_matrix[m][w])
        new_order = []
        while preferences:
            new_order.append(preferences.pop(0))  # his favorite
            if preferences:
                new_order.append(preferences.pop(len(preferences) // 2))  # middle one
        proposals_order.append(new_order)

    free_men = list(range(num_m))
    while free_men:
        m = free_men.pop(0)
        
        while proposals_order[m]:
            w = proposals_order[m].pop(0)

            # 如果女生w还未匹配或者她更喜欢新的匹配者，并且她也接受这个男生
            if (matched_women[w] == -1 or w_to_m_score_matrix[w][m] > w_to_m_score_matrix[w][matched_women[w]]) and m in W_dict[w]:
                if matched_women[w] != -1:
                    free_men.append(matched_women[w])

                # 更新匹配
                matched_women[w] = m
                matched_men[m] = w
                break

            # 否则男生被拒绝，继续下一个选择
            else:
                continue

    return matched_men

print(gale_shapley_modified_v2(m_to_w_score_matrix, w_to_m_score_matrix, M_dict, W_dict))

[51, 5, 43, 24, 60, 65, 4, 38, 48, 28, 27, 44, 37, 32, 22, 9, 40, 6, 47, 11, 13, 0, 53, 45, 56, 55, 31, 57, 33, 1, 67, 21, 20, 49, 36, 7, 12, 64, 2, 63, 26, 34, 61, 41, 23, 8, 46, 62, 15, 54, 52, 30, 25, 14, 29, 50, 35, 16, 58, 59, 39, 17, 66, 18, 19, 3, 10, 42]
